## NEURAL NETWORKS FINAL PROJECT

In [4]:
import csv
import re 
import tensorflow as tf
import numpy as np

from collections import Counter

from keras.layers import Dense, Activation
from keras.models import Sequential

# from tensorflow.keras import layers, losses
# from tensorflow.keras.layers import Dense, Flatten, Reshape
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import LeakyReLU

from keras import layers, losses
from keras.layers import Dense, Flatten, Reshape
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU

import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from datetime import datetime

import keras.callbacks
from keras.callbacks import TensorBoard
#%load_ext tensorboard

`CREATING 0-1 VECTORS FOR INGREDIENTS`

In [5]:
# importing original csv file 
datafile = open('archive/RecipeNLG_dataset.csv', 'r')
datareader = csv.reader(datafile, delimiter=',')
originalData = []
for row in datareader:
    originalData.append(row)    

for item in originalData[1]:
    print (item + "\n")

0

No-Bake Nut Cookies

["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]

["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]

www.cookbooks.com/Recipe-Details.aspx?id=44874

Gathered

["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]



In [6]:
# formatting first 1,000,000 data values to just be recipe title + ingredient list, separated by ';'

titleIngred = []

for row in originalData:
#for row in originalData[:1000000]:
    titleIngred.append(row[1] + " ; " + row[6])
        
titleIngred.pop(0);
        
print(titleIngred[0:4])

['No-Bake Nut Cookies ; ["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]', 'Jewell Ball\'S Chicken ; ["beef", "chicken breasts", "cream of mushroom soup", "sour cream"]', 'Creamy Corn ; ["frozen corn", "cream cheese", "butter", "garlic powder", "salt", "pepper"]', 'Chicken Funny ; ["chicken", "chicken gravy", "cream of mushroom soup", "shredded cheese"]']


In [7]:
# list of all ingredients, repeats included

ingredListMajor = []

for ingred in titleIngred:
    ingredString = ingred.split(" ; ")[1]
    regex = re.compile('[^a-zA-Z| |,-./\s]')
    temp = regex.sub('', ingredString).split(", ")
    for item in temp:
        if len(item)>2:
            ingredListMajor.append(item.lower())

print(ingredListMajor[0:100])
print("LENGTH: ", len(ingredListMajor))

['brown sugar', 'milk', 'vanilla', 'nuts', 'butter', 'bite size shredded rice biscuits', 'beef', 'chicken breasts', 'cream of mushroom soup', 'sour cream', 'frozen corn', 'cream cheese', 'butter', 'garlic powder', 'salt', 'pepper', 'chicken', 'chicken gravy', 'cream of mushroom soup', 'shredded cheese', 'peanut butter', 'graham cracker crumbs', 'butter', 'powdered sugar', 'chocolate chips', 'baking potatoes', 'extra lean ground beef', 'butter', 'milk', 'salt', 'pepper', 'cheddar cheese', 'bacon', 'green onion', 'sour cream', 'sugar', 'butter', 'egg', 'buttermilk', 'flour', 'salt', 'soda', 'buttermilk', 'rhubarb', 'vanilla', 'cream-style corn', 'whole kernel corn', 'crackers', 'egg', 'butter', 'pepper', 'tomatoes', 'water', 'onions', 'worcestershire sauce', 'green peppers', 'oil', 'pineapple', 'condensed milk', 'lemons', 'pecans', 'graham cracker crusts', 'dark sweet pitted cherries', 'ginger ale', 'flavor gelatin', 'boiling water', 'almond extract', 'marshmallows', 'powdered sugar', 'b

In [8]:
# list of 1,000 most common ingredients

ingredients = []

for item in Counter(ingredListMajor).most_common(1001):
    ingredients.append(item[0])
    
ingredients.remove('eggs')
    
print(ingredients)
print("LENGTH: ", len(ingredients))

['salt', 'sugar', 'butter', 'flour', 'onion', 'garlic', 'milk', 'water', 'vanilla', 'olive oil', 'pepper', 'brown sugar', 'egg', 'tomatoes', 'baking powder', 'lemon juice', 'cinnamon', 'sour cream', 'cream cheese', 'baking soda', 'celery', 'margarine', 'chicken', 'cheddar cheese', 'parsley', 'vegetable oil', 'onions', 'mayonnaise', 'parmesan cheese', 'oil', 'pecans', 'potatoes', 'kosher salt', 'carrots', 'nuts', 'soy sauce', 'black pepper', 'pineapple', 'honey', 'bacon', 'thyme', 'chicken broth', 'oregano', 'ground beef', 'mustard', 'unsalted butter', 'worcestershire sauce', 'mushrooms', 'cornstarch', 'paprika', 'powdered sugar', 'green pepper', 'vinegar', 'red pepper', 'green onions', 'lemon', 'nutmeg', 'ground cinnamon', 'shortening', 'walnuts', 'chicken breasts', 'rice', 'basil', 'heavy cream', 'buttermilk', 'chili powder', 'raisins', 'clove garlic', 'cocoa', 'cheese', 'red onion', 'ginger', 'orange juice', 'all-purpose', 'white sugar', 'soda', 'tomato sauce', 'bread', 'white wine',

In [9]:
# double list of zeros + ones, 1 representing ingredient is present in recipe
# if recipe has less than 5 of the most common ingredients, it is tossed

ingredientVectors = [[]]

ingredientVectors.append(ingredients)

for index in range(len(titleIngred)):
    counter = []
    for ingred in ingredients:
        if titleIngred[index].count(ingred)>0:
            counter.append(1)
        else:
            counter.append(0)
    if counter.count(1)>=5:
        ingredientVectors.append(counter)

In [9]:
# print(ingredientVectors[:5])

In [10]:
# numpy array of ingredient vectors created above 

npIngredients = np.array(ingredientVectors[2:])
print(npIngredients)
print(npIngredients.shape)

[[0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1997905, 1000)


In [11]:
print(npIngredients.shape)

(1997905, 1000)


`INGREDIENTS + STEPS`

In [66]:
# create list of ingredients + directions properly formatted
# later will turn this into a string and run through LSTM

ingredSteps = []

for line in originalData:
    new = line[3].split('", ')
    regex = re.compile('[^a-zA-Z| |, | 0-9]')

    steps = []

    for part in new:
        steps.append(regex.sub('', part))
    
    counter = 1
    stepsFinal = ""

    for part in steps:
        temp = str(counter) + ". " + part
        stepsFinal = stepsFinal + temp + "\n"
        counter=counter+1
    
    line2 = line[6]
    regex = re.compile('[^a-zA-Z| |,]')
    temp = regex.sub('', line2).split(", ")

    tempIngred = ""

    for index in range(len(temp)):
        if index==0:
            tempIngred = temp[0]
        else: tempIngred += ", " + temp[index]

    final = ("INGREDIENTS: " + tempIngred + "\n\n" + stepsFinal).rstrip(stepsFinal[-1])
    ingredSteps.append(final)

for line in ingredSteps[1:10]:
    print(line)
    print("\n\n")

INGREDIENTS: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits

1. In a heavy 2quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine
2. Stir over medium heat until mixture bubbles all over top
3. Boil and stir 5 minutes more Take off heat
4. Stir in vanilla and cereal mix well
5. Using 2 teaspoons, drop and shape into 30 clusters on wax paper
6. Let stand until firm, about 30 minutes



INGREDIENTS: beef, chicken breasts, cream of mushroom soup, sour cream

1. Place chipped beef on bottom of baking dish
2. Place chicken on top of beef
3. Mix soup and cream together pour over chicken Bake, uncovered, at 275u00b0 for 3 hours



INGREDIENTS: frozen corn, cream cheese, butter, garlic powder, salt, pepper

1. In a slow cooker, combine all ingredients Cover and cook on low for 4 hours or until heated through and cheese is melted Stir well before serving Yields 6 servings



INGREDIENTS: chicken, chicken gravy, cream of mushroom soup, shredded

`CREATING GAN MODEL`

In [12]:
def build_generator(input_shape, latent_dim):
    generator = Sequential()
    generator.add(Dense(100, input_dim=latent_dim, activation = 'relu')) 
    generator.add(Dense(200, activation = 'relu'))
    generator.add(Dense(input_shape, activation = 'relu'))
    return generator

In [13]:
def build_discriminator(input_shape):
    discriminator = Sequential()
    #discriminator.add(Flatten(input_shape=input_shape))
    discriminator.add(Dense(100, input_dim=input_shape))
    discriminator.add(LeakyReLU(alpha=0.01))
    discriminator.add(Dense(1, activation='sigmoid'))
    return discriminator

In [14]:
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [15]:
input_shape = (len(npIngredients[0]))
print(input_shape)
latent_dim = 100

discriminator = build_discriminator(input_shape)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

discriminator.trainable = False
generator = build_generator(input_shape, latent_dim)
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

1000


In [17]:
storeloss = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):
    
    # Attempt to load data 
    X_train = npIngredients

    # Labels
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # Training loop
    for iteration in range(iterations):
        
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        ingred = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_ingred = generator.predict(z)
        
        d_loss_acc_real = discriminator.train_on_batch(ingred, real)
        d_loss_acc_fake = discriminator.train_on_batch(gen_ingred, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_acc_real, d_loss_acc_fake)
        
        z = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_ingred = generator.predict(z)
        g_loss = gan.train_on_batch(z, real)

        if iteration % sample_interval == 0:

            storeloss.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration)

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (iteration, d_loss, 100.0 * accuracy, g_loss))

In [18]:
tf.config.run_functions_eagerly(True)

In [19]:
iterations = 10000
batch_size = 128
sample_interval = 50

train(iterations, batch_size, sample_interval)

/Users/jreyes/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


0 [D loss: 0.790043, acc.: 30.08%] [G loss: 0.696553]
50 [D loss: 0.526582, acc.: 84.77%] [G loss: 1.077079]
100 [D loss: 0.570516, acc.: 80.86%] [G loss: 1.079701]
150 [D loss: 0.635180, acc.: 72.27%] [G loss: 1.630250]
200 [D loss: 0.558507, acc.: 82.42%] [G loss: 1.237824]
250 [D loss: 0.467999, acc.: 85.94%] [G loss: 1.336222]
300 [D loss: 0.516354, acc.: 70.31%] [G loss: 0.817791]
350 [D loss: 0.407935, acc.: 89.06%] [G loss: 0.959550]
400 [D loss: 0.371440, acc.: 82.42%] [G loss: 2.294464]
450 [D loss: 0.417849, acc.: 90.62%] [G loss: 0.847445]
500 [D loss: 0.242848, acc.: 94.53%] [G loss: 1.547945]
550 [D loss: 0.358196, acc.: 87.50%] [G loss: 1.567303]
600 [D loss: 0.241266, acc.: 97.66%] [G loss: 1.791139]
650 [D loss: 0.181135, acc.: 93.75%] [G loss: 3.148202]
700 [D loss: 0.202648, acc.: 92.58%] [G loss: 2.881969]
750 [D loss: 0.292753, acc.: 91.02%] [G loss: 1.370440]
800 [D loss: 0.218847, acc.: 93.36%] [G loss: 2.265646]
850 [D loss: 0.261426, acc.: 92.58%] [G loss: 2.196

7250 [D loss: 0.076591, acc.: 96.88%] [G loss: 5.100473]
7300 [D loss: 0.182363, acc.: 92.19%] [G loss: 5.138851]
7350 [D loss: 0.185800, acc.: 93.36%] [G loss: 4.721430]
7400 [D loss: 0.158133, acc.: 96.09%] [G loss: 5.608827]
7450 [D loss: 0.068761, acc.: 98.05%] [G loss: 4.600836]
7500 [D loss: 0.062399, acc.: 97.66%] [G loss: 3.938398]
7550 [D loss: 0.104988, acc.: 94.92%] [G loss: 4.849719]
7600 [D loss: 0.110450, acc.: 95.70%] [G loss: 5.689035]
7650 [D loss: 0.075973, acc.: 98.05%] [G loss: 4.401589]
7700 [D loss: 0.084265, acc.: 96.88%] [G loss: 5.559424]
7750 [D loss: 0.063395, acc.: 98.44%] [G loss: 4.130835]
7800 [D loss: 0.141095, acc.: 96.09%] [G loss: 4.524290]
7850 [D loss: 0.364170, acc.: 86.72%] [G loss: 4.162910]
7900 [D loss: 0.137679, acc.: 96.09%] [G loss: 6.183698]
7950 [D loss: 0.233755, acc.: 92.19%] [G loss: 3.753822]
8000 [D loss: 0.420432, acc.: 89.45%] [G loss: 5.893247]
8050 [D loss: 0.055247, acc.: 98.05%] [G loss: 6.016898]
8100 [D loss: 0.169243, acc.: 9

In [63]:
latent_space_vector = np.random.normal(0, 1, (1, latent_dim))
arr = generator.predict(latent_space_vector)[0]

# count = np.count_nonzero(arr>0.5)
# for item in np.nditer(arr):
#     if item>0.5:
#         print(item)
# print('Total occurences of ">0.5" in array: ', count)

index = np.where(arr>0.5)
if index==0:
    print("none")
for num in index[0]:
    print(ingredientVectors[1][num])

salt
sugar
flour
vanilla
egg
baking soda
nuts
soda


In [64]:
latent_space_vector = np.random.normal(0, 1, (1, latent_dim))
arr = generator.predict(latent_space_vector)[0]

seed = "INGREDIENTS: "

for index in range(len(arr)):
    if arr[index]>0.5:
        if len(seed)<15:
            seed = seed + ingredientVectors[1][index]
        else: 
            seed = seed + ", " + ingredientVectors[1][index]

seed = seed + "\n"       

if len(seed) < 76:
    seed = seed + ((76-len(seed)) * " ")
        
seed = seed + "\n1. "
        
print(seed)

INGREDIENTS: salt, sugar, flour, milk, vanilla, egg, baking soda, oil, nuts, soda, cream

1. 


`RNN CODE`

In [71]:
masterString = "\n\n".join(ingredSteps)

In [ ]:
chars = sorted(list(set(masterString)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 80
step = 40
sentences = []
for recipe in ingredSteps[0:200]:
    for i in range(0, len(recipe) - seqlen - 1, step):
        sentences.append(recipe[i: i + seqlen + 1])


x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1



model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(masterString) - seqlen - 1)
    
    
    for diversity in [0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = masterString[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
           
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=250,
          callbacks=[print_callback])

Epoch 1/250
11/11 [==============================] - 3s 277ms/step - loss: 3.5434 - categorical_crossentropy: 3.5434 - accuracy: 0.1186

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "er, milk, broccoli, cheddar cheese

1. Melt butter in large soup pot and add oni"
er, milk, broccoli, cheddar cheese

1. Melt butter in large soup pot and add oni l
ou  h n  ioc  , o nnaa eed  lu i   a i e     nltu       se    i gn n  e in a  h hi   . in  arp r cg o  so f m  cs   n r  r   r n    in   oao   r  a   n i   d n  ma  l irn  bd u  ah.  r n e aga,e, lu c   e   c i n     aa      gn    c   n  it  par c    ie  ao  t   , s     aio    ami  r  n  a ls ia     amd i   in  k seeru u a    a  nos:  t  erudi  e ao  un  i a, da    
a,   ni st  g s    go   a   
----- diversity: 1.0
----- Generating with seed: "er, milk, broccoli, cheddar cheese

1. Melt butter in large soup pot and add oni"
er, milk, broccoli, cheddar cheese

1. Melt butter in large soup pot and add onig

8 Mit l ser urall pepat, thed chdugh mols, salond gat
, sd crarded tief, se wttl
3. Add lein aihe
4. Mthppemcwecheand in c5 peemifoum ant
5. malk broumeat ola ailn,as
8. Bat on 42 pave minteay susthrd
y wagton omlple a min tile

2. Pouto enafrapfo
Epoch 7/250
11/11 [==============================] - 3s 277ms/step - loss: 2.1045 - categorical_crossentropy: 2.1045 - accuracy: 0.3752

----- Generating text after Epoch: 6
----- diversity: 0.5
----- Generating with seed: "chicken, green chilies, salsa, sour cream

1. Beat eggs in large bowl
2. Stir in"
chicken, green chilies, salsa, sour cream

1. Beat eggs in large bowl
2. Stir in butt
4. Add mint
2. Pour in our on thea lour fout br and suat foll cok and butt r min lir fow wath ard co ld mix for unt llll wat, be ond bour  in and and butt r min to sur, m tor soin fol wnto and and on in bour bill untor, will on bur we f or in fous on bour, bater, buttor in foom water
5. Sour with pin tol boon milk, for cout and ove 8in f ook mill bater and m

2. Preheat the ovens broiler and set the ove preat et breamingaspare with add canze chllup vinNillaond on one
, Droaned atd bombiningr in and cheese in redires
in ungillogas, bakela dlmsturile and band blabonr dry wremet hucbs breama ce cell wilk, calever tog
4. Bry Jut ingr mixs, boalinh bumbed wime pactered will crocker ix ous dd wizz . Besh coll crease sml
4. Stil topey wiplle ctoschrramed checole nelly to her
5. Minuter
2. Serr chabore
Epoch 13/250
11/11 [==============================] - 3s 261ms/step - loss: 1.5207 - categorical_crossentropy: 1.5207 - accuracy: 0.5512

----- Generating text after Epoch: 12
----- diversity: 0.5
----- Generating with seed: "pasta pot while blending if needed
3. When the pasta is al dente, the shrimp wil"
pasta pot while blending if needed
3. When the pasta is al dente, the shrimp will chicken on beef and chick on butter
3. Brine breac pot the to a co cheat bour in sugar in corking bread and butter and butter
2. Add or until the pore and stor 10 min

6. Bake in a mushrut mare breas on on to shere
3. Add flour on top
----- diversity: 1.0
----- Generating with seed: "Vanilla, Orange Blossom Water, Fresh Rosemary, Orange, ubc

1. Note Youll need a"
Vanilla, Orange Blossom Water, Fresh Rosemary, Orange, ubc

1. Note Youll need at leave 
3. Bake it hwees Mix suger the creased meltet core souc
4. Add nel ingrediens uf fluts, cake mix until toggtred er thently
3. Put untol beet on well sor Add oil well Wotthre
3. When on tome more wher
6. Mix to crread chersed baking elatind
2. Stir and cunter milk croum sour preel on op stor mix rom 10 minuten maly in tienur
1. Stir whes

1. Bour in o lace pees
5. Layer well in but 
3. Ple
Epoch 19/250
11/11 [==============================] - 3s 259ms/step - loss: 1.1813 - categorical_crossentropy: 1.1813 - accuracy: 0.6478

----- Generating text after Epoch: 18
----- diversity: 0.5
----- Generating with seed: " minutes, without stirring, then stir 15 minutes
10. Pour into a cardboard box, "
 minutes, wi

2. Stir in bread mix and butter with cheese and pepper

1. Mix all ingredients together together a sauted cream of melted caken mix, butter and peanut butter
2. Add the cinseadle and shir with a flour heat and to meater of betted to pineapple
6. Add to serve over lemon brown
6. Cook until and 
----- diversity: 1.0
----- Generating with seed: ". At this point the dish can be cooled completely, covered, and kept in the refr"
. At this point the dish can be cooled completely, covered, and kept in the refring ther for 1 tip
12. Combine wets and colld
2. In beef, egg in pan
4. Pour in bisc
2. Mix gream chlese anionely
8. A
9. Tape 4. Namy do pook yourr, egg, togaton, tore maring sodened, tornumbr, kes and gelle chill egg

1. Top to ungrees
4. Droin for 15 to 2 heaves
4. Serve in peanut
6. Cover until heat Combine hast for 30 minutes
3. Pour on becoms
3. Spread in copride be fork
4. Add leatond corni
Epoch 25/250
11/11 [==============================] - 3s 258ms/step - loss: 0.9833 - categor

...

. Add the flour mixture to the egg mixture and stir together gently by hand with hoodles
3. Bake a for 1 monina
4. Laseroom hond Genale and reast and bake all jurcestly sutiy to serve oif in size slated boald filltalis in cream cheese unfor 
5. Bake on oup outcuppla on 1 bowl all of spuffing an orace and pan freeze, soups with large could Beat until thick
8. Bring to boil mixture over carrots and them in cuke rine peanuis powder
3. Add tho anstant 2 habe water until medium ss
Epoch 112/250
11/11 [==============================] - 3s 256ms/step - loss: 0.3126 - categorical_crossentropy: 0.3126 - accuracy: 0.9017

----- Generating text after Epoch: 111
----- diversity: 0.5
----- Generating with seed: "n butter under medium heat, stirring constantly until it becomes a smooth mixtur"
n butter under medium heat, stirring constantly until it becomes a smooth mixture in Smain and cream yogult and flour
3. Add nuts
4. Pour into a 13 x 9inch pan that hamburgerine in cherries
4. For 10 to 15 min

2. Combine all ingredients together and teaten least with flake shells on top of rine to cool quickly, then drain well Add dressing to broccoli mixture and cook until onions and bell peppers becover and ready doure pe
----- diversity: 1.0
----- Generating with seed: " cakes and arrange on a baking tray Cover with plastic wrap and chill for 12 hou"
 cakes and arrange on a baking tray Cover with plastic wrap and chill for 12 hour and stir while spoon and slicessin loaf in simat 3 to 4000004 mold 1 munt butter, pecans
4. Add sugar, baking soda and salt together
2. Let sly boiling reat
9. Beat until swiff 4 minutes or until key bowl dish Bowd vanilla
4. Stir sweel apples weft6. Cool Wher in cherries make
6. Easanwe at astid about and Bite and boil Rol time
6. Spread on pienfry top
3. Combine flour all or chisp

1. Beat egg
Epoch 118/250
11/11 [==============================] - 3s 261ms/step - loss: 0.3053 - categorical_crossentropy: 0.3053 - accuracy: 0.9044

----- Generating text after Ep

ENTS: oil, onion, stalks celery, celery soup, water, pork chops, bell pepper, mushrooms, onions and peppers, sour cream, water, green onions, ground beef, tomato sauce, salt

1. Put in beef
3. Makes after each egg to mixture
5. Mix quickly add then ceatered place and sugar
2. Add fruit and water until tender
3. Add the consomatoes and top at anserolaro2 hat for 10 to 20 minutes before set
3. Sprinkle into all of the wehen
9. Remove from heat and stir in jello
4. Add sugar and
----- diversity: 1.0
----- Generating with seed: "ENTS: oil, onion, stalks celery, celery soup, water, pork chops, bell pepper, mu"
ENTS: oil, onion, stalks celery, celery soup, water, pork chops, bell pepper, mushrooms, vegetable oil, eggs, salt

1. Prepurectuch with heat Combine the rise into 2 pastr5 vingrefiens after thice has beof nut more fien potter Groil oft reckaniog
4. Drain juice aster each egg until tender
4. Marsish
2. Cook for macorin the 350u00b0 oven to 340u00b0 for 15 to 30 minutes or lightly brow

11/11 [==============================] - 3s 267ms/step - loss: 0.2984 - categorical_crossentropy: 0.2984 - accuracy: 0.9046

----- Generating text after Epoch: 128
----- diversity: 0.5
----- Generating with seed: "dients to a serving bowl and toss together well Eat immediately

INGREDIENTS: Bl"
dients to a serving bowl and toss together well Eat immediately

INGREDIENTS: Black pepper, salt

1. Cook squashing and vanilla
4. Beat until smooth
2. Add gares microoa and and flour, salt and let stand overnight
5. Spread ollopered cooked mix well
4. Pour into a 13 x 9inch pan that has beet before do groun over fold in conferole softed
5. Remove finger or 1 cookie cherrins for 3 to 4 minutes or until onion and beef in skillet
8. Turn and toss of cooking then add to butter f
----- diversity: 1.0
----- Generating with seed: "dients to a serving bowl and toss together well Eat immediately

INGREDIENTS: Bl"
dients to a serving bowl and toss together well Eat immediately

INGREDIENTS: Blake placets

5. These chipped by leas a diple 6 hours patse can be us into fillstice and semanill
3. Pour over medium ingredients, except cheese
2. Spread in hast 5 minutes then with a dor cookirg
5. heat with lem, them
10. Salt Mish a wry pum copling unculvery sugar
5. Twish
7. Pour f
Epoch 135/250
11/11 [==============================] - 3s 261ms/step - loss: 0.2770 - categorical_crossentropy: 0.2770 - accuracy: 0.9130

----- Generating text after Epoch: 134
----- diversity: 0.5
----- Generating with seed: "rowned Cook for an additional 4 or 5 minutes to further brown and to reduce mois"
rowned Cook for an additional 4 or 5 minutes to further brown and to reduce moistenemastine icco baced
7. Put in mbifd mixture
5. Sart Combine water to constantly pudding, water and water eggs stir into meat balls Refrigerath

In [ ]:
for diversity in [0.1,0.5,1]:
    sentence = seed[:80]
    print(sentence)
    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
    print('\n')
    print('\n')